## keras の実装を tf.keras に置き換えて試してみる
- Reference
  - https://github.com/optuna/optuna/blob/master/examples/keras_simple.py

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import optuna

In [10]:
N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10

In [13]:
def objective(trial):
    keras.backend.clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    input_shape = (img_x, img_y, 1)

    model = keras.models.Sequential()
    model.add(
        keras.layers.Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2]),
            activation=trial.suggest_categorical("activation", ["relu", "linear"]),
            input_shape=input_shape,
        )
    )
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(CLASSES, activation="softmax"))

    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.RMSprop(lr=lr),
        metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    score = model.evaluate(x_valid, y_valid, verbose=0)

    return score[1]

In [14]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    #study.optimize(objective, n_trials=100, timeout=600)
    study.optimize(objective, n_trials=20, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")

    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2020-06-21 20:27:44,115] Finished trial#0 with value: 0.9079999923706055 with parameters: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'relu', 'lr': 0.0015081436515689158}. Best is trial#0 with value: 0.9079999923706055.
[I 2020-06-21 20:27:51,347] Finished trial#1 with value: 0.8349999785423279 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'linear', 'lr': 0.007351145207246328}. Best is trial#0 with value: 0.9079999923706055.
[I 2020-06-21 20:27:55,864] Finished trial#2 with value: 0.7900000214576721 with parameters: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear', 'lr': 3.288855338422068e-05}. Best is trial#0 with value: 0.9079999923706055.
[I 2020-06-21 20:28:03,744] Finished trial#3 with value: 0.8460000157356262 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'linear', 'lr': 4.004496921556263e-05}. Best is trial#0 with value: 0.9079999923706055.
[I 2020-06-21 20:28:09,087] Fin

Number of finished trials: 20
Best trial:
  Value: 0.9509999752044678
  Params: 
    filters: 64
    kernel_size: 5
    strides: 1
    activation: relu
    lr: 0.002521530006860772
